# 1. Read Dataset(Target Stories)

In [274]:
import glob
file_names = glob.glob("/Users/ibrahim/Desktop/SummerGR/Preprocess/query/*.txt")
raw_documents=[]
for file in file_names:
    try:
        with open(file, "r", encoding="utf-8") as f: raw_documents.append(f.read())
    except:
        pass
print("Number of documents: ",len(raw_documents))

Number of documents:  18


In [275]:
#raw_documents

##  1.1 Read Dataset(Query Story)

In [276]:
querydoc=glob.glob("/Users/ibrahim/Desktop/SummerGR/Preprocess/AllStories/*.txt")
query_documents=[]
for file in querydoc:
    try:
        with open(file, "r", encoding="utf-8") as f: query_documents.append(f.read())
    except:
        pass
query = query_documents[0].lower()
#print("Number of documents: ",len(query_documents))

# Print the Query

In [277]:
query_documents

['When it Was the Ninth Night,\n\n\x0cShe said, It hath reached me, O auspicious King, that when the\nyoung woman, the sorceress, took in hand some of the tarn water\nand spake over it words not to be understood, the fishes lifted\ntheir heads and stood up on the instant like men, the spell on\nthe people of the city having been removed. What was the lake\nagain became a crowded capital; the bazars were thronged with\nfolk who bought and sold; each citizen was occupied with his own\ncalling and the four hills became islands as they were whilome.\nThen the young woman, that wicked sorceress, returned to the King\nand (still thinking he was the negro) said to him, O my love!\nstretch forth thy honoured hand that I may assist thee to rise."\n"Nearer to me," quoth the King in a faint and feigned tone. She\ncame close as to embrace him when he took up the sword lying hid\nby his side and smote her across the breast, so that the point\nshowed gleaming behind her back.']

# 1.1 Locality Sensitive Hashing (LSH) Algorithm (First Layer)
https://nbviewer.org/github/bassimeledath/quora_profile/blob/master/questions_analysis.ipynb

https://www.pinecone.io/learn/locality-sensitive-hashing/

In [278]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

In [279]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def preprocess(text):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    tokens = text
    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in tokens:
        if ele in punc:
            tokens = tokens.replace(ele, "")
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [280]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    #print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest

In [281]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    result={}
    result = database.iloc[idx_array]
    
    #print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

In [282]:
def LSH_Candiadates_List(candidates_list):
    db = pd.DataFrame(candidates_list)#We have 67 Sentences
    db=db[0]
    #print(db)
    #Number of Permutations
    permutations = 2000
    #Number of Recommendations to return
    forest = get_forest(db, permutations)
    num_recommendations = 18
    #query = query_documents[0].lower()
    result = predict(query, db, permutations, num_recommendations, forest)
    #print('\n Top similar sentences \n', result1)


    #Create a list of candiates to be taken to the next layer which is Cosine Simialrity
    candidates=[]
    candidates=result.values
    return candidates

# Print a the candidates list from LSH

In [283]:
LSH_Candidates= LSH_Candiadates_List(raw_documents)

In [284]:
print(len(LSH_Candidates))

17


# Update Cosine Similarity
https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/

In [285]:
#Tokenize the candidates to be used in Bert Vector model
import nltk
from nltk.tokenize import word_tokenize
tokenized_sent = []
for s in LSH_Candidates:
    tokenized_sent.append(word_tokenize(s.lower()))

In [286]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np

[nltk_data] Downloading package punkt to /Users/ibrahim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [287]:
def cosine(u, v):#Cosine Similarity Calculation
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [288]:
from sentence_transformers import SentenceTransformer #Vectorize the sentences using bert
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [289]:
sentence_embeddings = sbert_model.encode(LSH_Candidates)

In [290]:
query_vec = sbert_model.encode([query])[0]#Vectorize the query using bert 

In [291]:
Cosine_Candidates={} #This dicitonary to get the candidates and thier cosine similarity
for sent in LSH_Candidates:
    sim = cosine(query_vec, sbert_model.encode([sent])[0])
    #print("Sentence = ", sent, "; similarity = ", sim)
    Cosine_Candidates[sent]=[sim]

In [292]:
import operator#To sort the candidates from the highest to the lowest
sorted_d = dict(sorted(Cosine_Candidates.items(), key=operator.itemgetter(1),reverse=True))
print(sorted_d)

{'Then when it was the ninth night, SHE CONTINUED:\n\nI have heard, O auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city. It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state. The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up.’ In a low voice, the king replied: ‘Come to me.’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back. With another blow he cut her in two, and threw the two halves on the ground.\n\nWhen he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and thanking him. The

In [293]:
NW_Candidates2=[] #The questions that has 85% similarity only
for i in sorted_d:
    if sorted_d[i] >= [0.85]:
        NW_Candidates2.append(i)

In [294]:
NW_Candidates=NW_Candidates2[:10]

In [295]:
print(NW_Candidates[0])

Then when it was the ninth night, SHE CONTINUED:

I have heard, O auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city. It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state. The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up.’ In a low voice, the king replied: ‘Come to me.’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back. With another blow he cut her in two, and threw the two halves on the ground.

When he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and thanking him. The king 

In [296]:
res = len(query.split('.'))

In [297]:
res

6

In [300]:
from nltk.util import ngrams


n = 6
s2=NW_Candidates[0]
n_grams = ngrams(s2.split('.'), n)
SW=[]
for grams in n_grams :
    SW.append(grams)

In [314]:
SW[0]

('Then when it was the ninth night, SHE CONTINUED:\n\nI have heard, O auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city',
 ' It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state',
 ' The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up',
 '’ In a low voice, the king replied: ‘Come to me',
 '’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back',
 ' With another blow he cut her in two, and threw the two halves on the ground')

In [302]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
SW2=[]
for i in SW:
    reconstructedSentence=TreebankWordDetokenizer().detokenize(i)
    SW2.append(reconstructedSentence)

In [329]:
print(SW2[0])

Then when it was the ninth night, SHE CONTINUED:

I have heard, O auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city  It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state  The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up ’ In a low voice, the king replied: ‘Come to me ’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back  With another blow he cut her in two, and threw the two halves on the ground


In [321]:
SW2[1]

'It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state  The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up ’ In a low voice, the king replied: ‘Come to me ’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back  With another blow he cut her in two, and threw the two halves on the ground \n\nWhen he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and thanking him'

In [327]:
from minineedle import needle, core

alignment = needle.NeedlemanWunsch(query, SW2[0])
print(alignment.get_identity(),'\n')
print(alignment)

42.12 

Alignment of Query and Target Sentence is:
	-----when it was the ninth night,

she --sai---d-, it hath reached me, o auspicious king, that when the
young woman, the sorceress---, took in ha-nd -some of the tarn water
and spake- -over --i-t--------- words not to be- understood, the fish-----e-s lifted
their heads and stood up on -the instant like men, the spell on
the pe-ople- --of the city having been removed. -what was the- lake
again- -became a crowded capital; the bazars were thronged with
folk who bought and sold; each citizen -was occupied with his own
calling and the four hills became islands as they were whilome.
then the young wo-man, -that- wicked sorceress, re-t--ur-------ned to the king
and (sti--ll thinking he was the negro-) said to h-im, o my love!
str-etch forth thy honoured hand that i may assi-st thee --to r---ise."
"nearer to me," quoth the king --in --a fai--nt-- --a-n-----d feign--ed tone. she
came close- as to embr-ace him whe----n- he to--ok- up th----e s

In [257]:
from minineedle import needle, core
MaxNW={}
for i in SW2:
    alignment = needle.NeedlemanWunsch(query, i)
    x = alignment.get_identity()
    y = alignment 
    MaxNW[i]=[x , query, i, y]
    max_value = max(MaxNW.values())
print('Query: ',s1,'\n')
print('Target Sentence: \n',max_value[2],'\n')
print('Perecent Identity using NW:',max_value[0])

KeyboardInterrupt: 

# Needleman-Wunsch using Window Sliding
http://www.medcomp.medicina.unipd.it/main_site/doku.php?id=nessie:algorithm

https://scholarworks.rit.edu/cgi/viewcontent.cgi?referer=https://www.google.com/&httpsredir=1&article=1005&context=theses



https://arxiv.org/pdf/1701.06279.pdf page2

http://www.scielo.org.mx/scielo.php?pid=S1405-55462019000300999&script=sci_arttext&tlng=en

http://www.pertanika.upm.edu.my/resources/files/Pertanika%20PAPERS/JST%20Vol.%2030%20(1)%20Jan.%202022/06%20JST-2912-2021.pdf //this one is really nice one


https://www.tensorflow.org/text/api_docs/python/text/sliding_window code

In [23]:
#Query tokenization and normalization
#List1=[]
#for i in NW_Candidates:
    

In [67]:
import nltk
from nltk.tokenize import sent_tokenize
S1N = []
for s in NW_Candidates:
    print(s)
    S1N.append(sent_tokenize(s.lower()))

Then when it was the ninth night, SHE CONTINUED:

I have heard, O auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city. It became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state. The sorceress went straight away to the tomb and said to the king: ‘Give me your noble hand, my darling, and get up.’ In a low voice, the king replied: ‘Come to me.’ When she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back. With another blow he cut her in two, and threw the two halves on the ground.

When he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and thanking him. The king 

In [69]:
print(S1N[0])

['then when it was the ninth night, she continued:\n\ni have heard, o auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city.', 'it became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state.', 'the sorceress went straight away to the tomb and said to the king: ‘give me your noble hand, my darling, and get up.’ in a low voice, the king replied: ‘come to me.’ when she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back.', 'with another blow he cut her in two, and threw the two halves on the ground.', 'when he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and thanking

In [60]:
import nltk
from nltk.tokenize import sent_tokenize
S2N = []
for s in query_documents:
    S2N.append(sent_tokenize(s.lower()))

In [65]:
S1N[0]
#S2N[0]

['then when it was the ninth night, she continued:\n\ni have heard, o auspicious king, that after the sorceress had taken water from the pool and spoken some unintelligible words over it, the fish danced, lifted their heads and immediately rose up, as the magic spell was removed from the city.',
 'it became inhabited again, the merchants buying and selling and each man practising his craft, while the islands were restored to their former state.',
 'the sorceress went straight away to the tomb and said to the king: ‘give me your noble hand, my darling, and get up.’ in a low voice, the king replied: ‘come to me.’ when she did this he, with the drawn sword in his hand, struck her in the breast as she clung on to him, so that it emerged gleaming from her back.',
 'with another blow he cut her in two, and threw the two halves on the ground.',
 'when he came out he found the young man whom she had enchanted standing waiting for him, congratulating him on his escape, kissing his hand and than

# SW Example


In [220]:
s1="The religion is Islam."
s2="Saudi Arabia has 13 states and the offical religion is Islam and the head of states is amir. The capital city of Saudi Arabia is Riyadh."



In [221]:
#length of the source sentence(how many words)
res = len(s1.split())


In [222]:
res

4

In [223]:
#Now sliding window to divide the target sentence into 4 words per each index


In [224]:
from nltk.util import ngrams


n = 4

n_grams = ngrams(s2.split(), n)
SW=[]
for grams in n_grams :
    SW.append(grams)

In [225]:
SW[0]

('Saudi', 'Arabia', 'has', '13')

In [226]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
SW2=[]
for i in SW:
    reconstructedSentence=TreebankWordDetokenizer().detokenize(i)
    SW2.append(reconstructedSentence)

In [227]:
from minineedle import needle, core
MaxNW={}
for i in SW2:
    alignment = needle.NeedlemanWunsch(s1, i)
    x = alignment.get_identity()
    y = alignment 
    MaxNW[i]=[x , query, i, y]
    max_value = max(MaxNW.values())
print('Query: ',s1,'\n')
print('Target Sentence: \n',max_value[2],'\n')
print('Perecent Identity using NW:',max_value[0])

Query:  The religion is Islam. 

Target Sentence: 
 offical religion is Islam 

Perecent Identity using NW: 69.23
